# Tunisian Horses - Only stills
* do not use the cropped images, because the cropped images are most probably extracted from the stills

## Create metadata

In [ ]:
import shutil
import os
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical

In [ ]:

# 2. Prepare data for model training:
# Base path to the database
base_dir = '../data/THoDBRL2015'

metadata = []

# Consolidate images into a training directory
output_train_dir = os.path.join(base_dir, 'training_data')

os.makedirs(output_train_dir, exist_ok=True)

parts = [
    'Part1', 
    # 'Part2', 
    # 'Part3', 
    # 'Part4', 
    # 'Part5'
]

# Iterate through each Part folder
for part in parts:
    videos_dir = os.path.join(base_dir, part, 'videos')

    for horse_id_folder in os.listdir(videos_dir):
        horse_path = os.path.join(videos_dir, horse_id_folder)

        if os.path.isdir(horse_path):  # Ensure it's a directory
            # Find all stills folders (e.g., images, images1, images2, etc.)
            for stills_folder in os.listdir(horse_path):
                stills_path = os.path.join(horse_path, stills_folder)

                if os.path.isdir(stills_path) and stills_folder.startswith('images'):  # Check for folders named 'images*'
                    target_dir = os.path.join(output_train_dir, f'horse_{horse_id_folder}')

                    os.makedirs(target_dir, exist_ok=True)
                    
                    # Copy all image files from the stills folder to the target directory
                    for img_file in os.listdir(stills_path):
                        img_path = os.path.join(stills_path, img_file)

                        if img_file.endswith(('.jpg', '.jpeg', '.png')):  # Ensure it's an image file
                            metadata.append({
                                'horse_id': horse_id_folder,
                                'image_path': img_path,
                                'width': Image.open(img_path).size[0],
                                'height': Image.open(img_path).size[1],
                            })
                        
                            # shutil.copy(img_path, target_dir)

                    # print(f"Copied images from {stills_path} to {target_dir}")

# Create DataFrame with specified dtypes
metadata_df = pd.DataFrame(metadata, dtype='object').astype({
    'horse_id': 'int64',          # Assuming it's an integer
    'image_path': 'string',
    'width': 'int64',
    'height': 'int64',
})




## Descriptive Analysis

In [ ]:
print(f"Total images: {len(metadata_df)}")
print(f"Total horses: {metadata_df['horse_id'].nunique()}")
print(f"Width: {metadata_df['width'].unique()}")
print(f"Height: {metadata_df['height'].unique()}")


In [ ]:
print(metadata_df[:5])

In [ ]:
grouped_horse = metadata_df.groupby(['horse_id'], observed=True)

grouped_horse.describe()

In [ ]:
# Images per horse id
grouped_horse.size().agg(['min', 'max', 'mean'])

In [ ]:
df = pd.DataFrame(metadata_df)

## Prepare dataset

In [ ]:
# For splitting data
# Load data
# Prepare dataset
def load_images_and_labels(metadata_df):
    images = []
    labels = []
    
    for _, row in metadata_df.iterrows():
        if os.path.exists(row['image_path']):
            img = Image.open(row['image_path']).resize((128, 128))  # Scale to same size
            images.append(np.array(img))
            labels.append(row['horse_id'])  # use horse_id for the label
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels
    


In [ ]:
# Load data
images, labels = load_images_and_labels(metadata_df)


### Split in train and test

In [ ]:

# Split in train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
# Normalise pixelvalues
# X_train = X_train / 255.0
# X_test = X_test / 255.0

# Transfer labels to category data
# num_classes = len(metadata_df['horse_id'].unique())
num_classes = len(np.unique(labels))


y_train = y_train - 1  # If needed then custumize labels (sometimes gives error)
y_test = y_test - 1  # The same for test

# print(y_train.min())  # Min label value
# print(y_train.max())  # Max label value

# Convert labels to categorical data
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
# Normalise pixel values?

## Create CNN

In [ ]:
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model():
    model = Sequential()

    # Add layers step-by-step
    model.add(keras.Input(shape=(128, 128, 3)))
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # To prevent overfitting

    model.add(Dense(num_classes, activation='softmax'))  # Output layer
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


# Create a model
model = create_model()

# Model summary
model.summary()

In [ ]:
y_train.shape
y_train[:5]

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=2,
    batch_size=32,
    validation_data=(X_test, y_test)
)


Findings with epochs=10 and batch_size=32
- expected time (335 sec * 10 epochs) ~~ 1 hour
- Accuracy in 1st epoch: 0.54, and loss 3.92
- Accuracy in 2nd epoch: on the start already accuracy of 0.91
- In 2nd epoch, no big changes.

Findings with epochs=1 and batch_size=64
- expected time ~6 min (so same time per epoch)
- accuracy is >0.95.

Explanation:
- 

## Save model

In [ ]:
# Save the entire model as a `.keras` zip archive.
model.save('../data/saved_models/my_model_epoch_1_batch_size_128.keras')

## Confusion matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

class_names = [str(i) for i in range(metadata_df['horse_id'].nunique())]  

# predict the labels for testset
print(X_test.shape)
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)

# Plot The confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Class')
plt.ylabel('Real Class')
plt.title('Confusion Matrix')
plt.show()

## GradCAM
Use GradCAM to visualize the areas of the image that the model is looking at when making predictions.
We use the last convolutional layer of the model as the last layer before the output layer.
We use `tf-explain` to generate the GradCAM heatmap.

If we have had used 'pytorch', we should have used 'torchcam' for this, which is making use of the 'hooks' in pyTorch.

In [ ]:
# Load pretrained model or your own
# model = tf.keras.applications.vgg16.VGG16(weights="imagenet", include_top=True)
from tf_explain.core.grad_cam import GradCAM

IMAGE_PATH = df['image_path'][0]

# Load a sample image (or multiple ones)
img = keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(128, 128))
img = keras.preprocessing.image.img_to_array(img)
data = ([img], None)

# Start explainer
explainer = GradCAM()
images = np.expand_dims(img, axis=0) # Expand to 4D tensor
print(images.shape)
prediction = model(images)
class_index = np.argmax(prediction, axis=1)[0] # Get the predicted class / horse_id
grid = explainer.explain(data, model, class_index=class_index, layer_name="conv2d_2")

explainer.save(grid, ".", "grad_cam.png")

# Gives error:
# AttributeError: The layer sequential has never been called and thus has no defined output.
